<h1> <center> Clustering Misorientation: An example with Grain Boundaries </h1></center>

In [1]:
%matplotlib qt5

# Important external dependencies
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN

# Other external dependencies
#from sklearn.datasets import make_blobs

# Local dependencies (now conda version 0.1.1)
from orix.quaternion.orientation import Orientation, Misorientation
#from orix.quaternion.rotation import Rotation
from orix.quaternion.symmetry import D6#,C1, D6h,Oh
from orix.quaternion.orientation_region import OrientationRegion
#from orix.vector.neo_euler import AxAngle
#from orix.vector import Vector3d
from orix import plot

# Colorisation
from skimage.color import label2rgb
from matplotlib.colors import to_rgb, to_hex
#MPL_COLORS_RGB = [to_rgb('C{}'.format(i)) for i in range(10)]
#MPL_COLORS_HEX = [to_hex(c) for c in MPL_COLORS_RGB]

# Animation
#import matplotlib.animation as animation

# Visualisation
#from mpl_toolkits.mplot3d.art3d import Line3DCollection
#from matplotlib.patches import Circle
from matplotlib.lines import Line2D
#from matplotlib.collections import CircleCollection
#from scipy.spatial import ConvexHull
#from mpl_toolkits.mplot3d.art3d import Poly3DCollection

#plt.rc('font', size=6)

In [2]:
filepath = './data/Ti_orientations.ctf'
# Load data from CTF file
dat = np.loadtxt(filepath, skiprows=1)[:, :3]

ori = Orientation.from_euler(np.radians(dat)).reshape(381, 507)[-100:, :200].set_symmetry(D6)

fundamental_region = OrientationRegion.from_symmetry(D6, D6)

# Compute misorientations
misori_base = Misorientation(~ori[:, :-1] * ori[:, 1:])
boundary_mask = misori_base.angle > np.radians(7)
misori = misori_base[boundary_mask].set_symmetry(D6, D6)
print('Number of misorientations:', misori.size)

Number of misorientations: 860


In [3]:
small_mask = misori.angle < np.radians(7)

<center><h2> Creating a distance metric (skip to next heading to load from a file) </h2></center>

RAM smash

In [ ]:
confirm = input('Are you sure? (y/n) ')
if confirm == 'y':
    mismisori = (~misori).outer(misori)
    mismisori_equiv = D6.outer(~misori).outer(D6).outer(D6).outer(misori).outer(D6)
    distance = mismisori_equiv.angle.data.min(axis=(0, 2, 3, 5))

Should work

In [ ]:
confirm = input('Are you sure? (y/n) ')
if confirm == 'y':
    from itertools import combinations_with_replacement as icombinations
    from tqdm import tqdm_notebook
    distance = np.empty((misori.size, misori.size))

    for i, j in tqdm_notebook(list(icombinations(range(misori.size), 2))):
        m_1, m_2 = misori[i], misori[j]
        mismisori = D6.outer(~m_1).outer(D6).outer(D6).outer(m_2).outer(D6)
        d = mismisori.angle.data.min(axis=(0, 2, 3, 5))
        distance[i, j] = d
        distance[j, i] = d

<center> <h2> ... Or, here is one we made earlier </h2> </center>

In [4]:
filepath_2 = './data/misori-distance((100, 200)).npy'
distance = np.load(filepath_2)

Cluster

In [5]:
distance = distance[~small_mask][:, ~small_mask]

In [6]:
# Compute clusters
dbscan = DBSCAN(0.05, 10, metric='precomputed').fit(distance)
print('Cluster labels:', np.unique(dbscan.labels_))
n_clusters = len(np.unique(dbscan.labels_)) - 1
print('Number of clusters:', n_clusters)

# Generate colors
colors = [to_rgb('C{}'.format(i)) for i in range(10)]
c = label2rgb(dbscan.labels_, colors=colors)

Cluster labels: [-1  0  1  2  3]
Number of clusters: 4


In [7]:
misori = misori[~small_mask]

In [8]:
cluster_means = Misorientation.stack([misori[dbscan.labels_ == label].mean() for label in np.unique(dbscan.labels_)[1:]]).flatten()
cluster_means = cluster_means.set_symmetry(D6, D6)

In [9]:
np.degrees(cluster_means.angle.data)

array([57.32484832, 68.81388421, 31.58939741, 60.10115054])

In [10]:
# Generate map
mapping = np.ones(misori_base.shape + (3,))

mapping[np.where(boundary_mask)[0][~small_mask], np.where(boundary_mask)[1][~small_mask]] = c

<h2><center> Plotting our results </h2></center>

In [25]:
# Plot clustered misorientations
fig = plt.figure(figsize=(3.484252, 3.484252))
gridspec = plt.GridSpec(1, 1, left=0, right=1, bottom=0, top=1, hspace=0.05)


ax_misori = fig.add_subplot(gridspec[0], projection='axangle', aspect='equal', proj_type='ortho')
ax_misori.scatter(misori,s=4,c=c) 
ax_misori.plot_wireframe(fundamental_region, color='black', linewidth=0.5, alpha=0.1, rcount=361, ccount=361)


ax_misori.set_axis_off()
ax_misori.set_xlim(0.2, 1.2)
ax_misori.set_ylim(-.1, .9)
ax_misori.set_zlim(-0, 1)
ax_misori.view_init(90, -60)


handles = [
    Line2D(
        [0], [0], 
        marker='o', color='none', 
        label=i+1, 
        markerfacecolor=color, markersize=5
    ) for i, color in enumerate(colors[:n_clusters])
]

ax_misori.legend(handles=handles, loc='upper left')

In [27]:
fig = plt.figure(figsize=(3.484252*2, 1.5*2))
gridspec = plt.GridSpec(1, 1, left=0, right=1, bottom=0, top=1, hspace=0.05)

ax_misori = fig.add_subplot(gridspec[0], projection='axangle', proj_type='ortho', aspect='equal')
ax_misori.scatter(misori, c=c, s=4)
ax_misori.plot_wireframe(fundamental_region, color='black', linewidth=0.5, alpha=0.1, rcount=181, ccount=361)

ax_misori.set_axis_off()
ax_misori.set_xlim(0.1, 1.1)
ax_misori.set_ylim(0.1, 1.1)
ax_misori.set_zlim(-0, 1)
ax_misori.view_init(0, -60)

And to conclude, we plot our grain boundaries in real space

In [ ]:
fig = plt.figure(figsize=(3.484252, 2))

gridspec = plt.GridSpec(1, 1, left=0, right=1, bottom=0, top=1, hspace=0.05)
ax_mapping = fig.add_subplot(gridspec[0])
ax_mapping.imshow(mapping)

ax_mapping.set_xticks([])
ax_mapping.set_yticks([])